In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [ ]:
df_links = pd.read_csv("kdca_links.csv")
links = df_links["링크"].tolist()
print(f"총 {len(links)}개의 링크를 불러왔습니다.")


In [7]:
def extract_disease_info(url, disease_name):
    res = requests.get(
        url,
        headers={
            "User-Agent": "Mozilla/5.0",
            "Referer": "https://terms.naver.com/"
        },
        timeout=10
    )
    soup = BeautifulSoup(res.text, "html.parser")

    data = {
        "질병명": disease_name,
        "정의": "",
        "원인": "",
        "증상": "",
        "진단법": "",
        "치료방법": ""
    }

    headers = soup.select("h3")

    for h3 in headers:
        h3_text = h3.text.strip()
        content = ""

        # 다음 형제 요소들을 순회하며 텍스트 수집
        sib = h3.find_next_sibling()
        while sib and sib.name != "h3":
            content += sib.get_text(separator=" ").strip() + " "
            sib = sib.find_next_sibling()

        # 줄바꿈 기호 제거
        content = content.replace("\\n", "").replace("\n", "").strip()

        # 분기 처리
        if "개요-정의" in h3_text:
            data["정의"] = content
        elif h3_text == "개요" and not data["정의"]:
            data["정의"] = content
        elif "원인" in h3_text:
            data["원인"] += content + " "
        elif "증상" in h3_text:
            data["증상"] += content + " "
        elif "진단" in h3_text:
            data["진단법"] += content + " "
        elif "치료" in h3_text:
            data["치료방법"] += content + " "

    return data




In [8]:
all_data = []
for i, row in enumerate(df_links.itertuples()):
    try:
        info = extract_disease_info(row.링크, row.질병명)  # ✅ 인자 2개 전달
        all_data.append(info)
        print(f"[{i+1}/{len(df_links)}] 완료: {info['질병명']}")
        time.sleep(0.3)
    except Exception as e:
        print(f"[에러] {row.링크} - {e}")


[1/374] 완료: 가정 내 아동안전
[2/374] 완료: 간기능검사
[3/374] 완료: 간농양
[4/374] 완료: 간염
[5/374] 완료: 감기
[6/374] 완료: 감염단핵구증
[7/374] 완료: 감염성 질환의 혈청검사
[8/374] 완료: 감염성심내막염
[9/374] 완료: 갑상선 검사
[10/374] 완료: 갑상선 검사(갑상선 세포검사 및 조직검사)
[11/374] 완료: 갑상선 검사(방사성 요오드 섭취율)
[12/374] 완료: 갑상선 검사(양전자방출단층촬영(PET))
[13/374] 완료: 갑상선 검사(전산화단층촬영(CT)과 자기공명영상(MRI))
[14/374] 완료: 갑상선 검사(초음파)
[15/374] 완료: 갑상선 검사(혈액검사 및 요검사)
[16/374] 완료: 갑상선 결절
[17/374] 완료: 갑상선 양성 종양
[18/374] 완료: 갑상선기능항진증
[19/374] 완료: 거부반응과 면역억제제
[20/374] 완료: 건강검진(국가건강검진)
[21/374] 완료: 건강검진(암 검진)
[22/374] 완료: 건강기능식품
[23/374] 완료: 건강하게 염분 섭취 하는 방법! 알려드겠습니다!
[24/374] 완료: 건강하게 탄수화물 먹는 방법! 알려드리겠습니다!
[25/374] 완료: 건막염
[26/374] 완료: 건선
[27/374] 완료: 겨울철 대설대비 건강수칙
[28/374] 완료: 겨울철 대표 질환별 응급조치 요령
[29/374] 완료: 겨울철 동상 및 동창 예방요령
[30/374] 완료: 겨울철 저체온증 예방요령
[31/374] 완료: 겨울철 한파대비 건강수칙
[32/374] 완료: 결핵
[33/374] 완료: 고위험 임산부
[34/374] 완료: 고칼슘혈증
[35/374] 완료: 고혈당
[36/374] 완료: 고혈압
[37/374] 완료: 고혈압 환자의 운동요법
[38/374] 완료: 고혈압성 심질환
[39/374] 완료: 골수염
[40/374] 완료: 골절
[41/374] 완료: 공기 오염
[42/374] 완료: 공수병

In [21]:
# CSV 파일 불러오기
df = pd.DataFrame(all_data)

df = df.rename(columns={
    "질병명": "병명",
    "진단법": "진단",
    "치료방법": "치료"
})

# 1. '증상'이 NaN이거나
# 2. 공백, 줄바꿈 등만 있는 문자열인 경우 제거
df_cleaned = df[~(
    df["증상"].isna() |                       # NaN
    (df["증상"].astype(str).str.strip() == "")  # 공백
)]

# 결과 저장
df_cleaned.to_csv("kdca_disease_details_cleaned.csv", index=False, encoding="utf-8-sig")

print("정제 완료: '증상'이 없는 질병은 삭제되었습니다.")

정제 완료: '증상'이 없는 질병은 삭제되었습니다.
